In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data= keras.utils.image_dataset_from_directory('./Medicinal Leaf Images/',
                                                    image_size=(224,224),
                                                    batch_size=32,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    seed=42)

Found 1835 files belonging to 30 classes.
Using 1468 files for training.


In [3]:
test_data=keras.utils.image_dataset_from_directory('./Medicinal Leaf Images/',
                                                  image_size=(224,224),
                                                  batch_size=32,
                                                  validation_split=0.2,
                                                  subset='validation',
                                                  seed=42)

Found 1835 files belonging to 30 classes.
Using 367 files for validation.


In [4]:
train_output_dir = './train_dataset'
test_output_dir = './test_dataset'

os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)


In [5]:
def save_dataset_to_folder(dataset, output_dir):
    for images, labels in dataset:
        for i in range(images.shape[0]):
            image = images[i].numpy()
            label = labels[i].numpy()
            label_folder = os.path.join(output_dir, str(label))
            
            # Create label folder if it doesn't exist
            os.makedirs(label_folder, exist_ok=True)
            
            # Normalize and save image to label folder
            image = image / 255.0  # Normalize pixel values to the range [0, 1]
            image_filename = f'image_{i}.png'
            image_path = os.path.join(label_folder, image_filename)
            plt.imsave(image_path, image)


In [6]:
save_dataset_to_folder(train_data, train_output_dir)

In [7]:
save_dataset_to_folder(test_data, test_output_dir)

In [8]:
data_augmentation=ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    zoom_range=0.1)

In [9]:
base_model=keras.applications.VGG16(weights='imagenet',
                                         include_top=False,
                                         input_shape=(224,224,3))

In [10]:
num_classes=len(train_data.class_names)
model = keras.Sequential()

# Add the ResNet152 base
model.add(base_model)

# Add GlobalAveragePooling2D layer
model.add(layers.GlobalAveragePooling2D())

# Add Dense layer with, for example, 256 units and 'relu' activation
model.add(layers.Dense(256, activation='relu'))

# Add Dropout layer with a specified dropout rate, e.g., 0.5
model.add(layers.Dropout(0.5))

# You can add more Dense and Dropout layers as needed

# Output layer (assuming binary classification, change units for multi-class)
model.add(layers.Dense(units=num_classes, activation='softmax'))

# Compile the model (adjust the optimizer, loss, and metrics as needed)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 30)                7710      
                                                                 
Total params: 14853726 (56.66 MB)
Trainable params: 14853726 (56.66 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

In [ ]:
history=model.fit(train_data, epochs=15,validation_data=test_data)

Epoch 1/15


46/46 [==============================] - 57066s 1267s/step - loss: 5.3717 - accuracy: 0.0416 - val_loss: 3.3923 - val_accuracy: 0.0545
Epoch 2/15
46/46 [==============================] - 1872s 40s/step - loss: 3.3819 - accuracy: 0.0565 - val_loss: 3.3889 - val_accuracy: 0.0545
Epoch 3/15
46/46 [==============================] - 1765s 38s/step - loss: 3.3826 - accuracy: 0.0674 - val_loss: 3.7646 - val_accuracy: 0.0545
Epoch 4/15
46/46 [==============================] - 1775s 39s/step - loss: 3.3996 - accuracy: 0.0681 - val_loss: 3.3848 - val_accuracy: 0.0545
Epoch 5/15
46/46 [==============================] - 1743s 38s/step - loss: 3.3845 - accuracy: 0.0668 - val_loss: 3.3898 - val_accuracy: 0.0436
Epoch 6/15
46/46 [==============================] - 9966s 221s/step - loss: 3.3650 - accuracy: 0.0613 - val_loss: 3.3836 - val_accuracy: 0.0545
Epoch 7/15
36/46 [======================>.......] - ETA: 9:03 - loss: 3.3635 - accuracy: 0.0599

In [ ]:
model.save('model.h5')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()